In [1]:
import torch
from byol_pytorch import BYOL
from torchvision import models

/Users/feiwu/opt/anaconda3/envs/torch/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
IMG_SIZE = 220
BATCH_SIZE = 128
EPOCHS = 200
SEED = 0
SIMSIAM = False

resnet = models.resnet50(pretrained=True)

learner = BYOL(
    resnet,
    image_size = IMG_SIZE,
    hidden_layer = 'avgpool',
    use_momentum = not SIMSIAM,
)

/Users/feiwu/opt/anaconda3/envs/torch/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/Users/feiwu/opt/anaconda3/envs/torch/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
x = torch.randn(2, 3, IMG_SIZE, IMG_SIZE)
image_one, image_two = learner.augment1(x), learner.augment2(x)

In [6]:
online_proj_one, _ = learner.online_encoder(image_one)
online_proj_two, _ = learner.online_encoder(image_two)

online_pred_one = learner.online_predictor(online_proj_one)
online_pred_two = learner.online_predictor(online_proj_two)

with torch.no_grad():
    target_encoder = learner._get_target_encoder() if learner.use_momentum else learner.online_encoder
    target_proj_one, _ = target_encoder(image_one)
    target_proj_two, _ = target_encoder(image_two)
    target_proj_one.detach_()
    target_proj_two.detach_()

In [7]:
target_proj_one.shape, target_proj_two.shape

(torch.Size([2, 256]), torch.Size([2, 256]))

In [41]:
import torch.nn.functional as F
def loss_fn(x, y):
    x = F.normalize(x, dim=-1, p=2)
    y = F.normalize(y, dim=-1, p=2)
    return 2 - 2 * (x * y).sum(dim=-1)

def loss_fn(x, y):
    x = F.normalize(x, dim=-1, p=2)
    y = F.normalize(y, dim=-1, p=2)
    return 2 - 2 * torch.matmul(x, y.T)

In [51]:
loss_one = loss_fn(online_pred_one, target_proj_two.detach()) # shape: (batch_size, projection_size)
loss_two = loss_fn(online_pred_two, target_proj_one.detach())
weight = torch.tensor([[1, 0], [0, 1]])
loss = weight * (loss_one + loss_two) # shape: (batch_size, batch_size)
loss = loss.sum(dim=-1) # the reduction is BYOL is the same as simCLR
loss = loss.mean()

In [52]:
loss

tensor(3.9432, grad_fn=<MeanBackward0>)

In [40]:
loss_one, loss_two, loss

(tensor([2.1195, 1.8425], grad_fn=<RsubBackward1>),
 tensor([1.9155, 2.0088], grad_fn=<RsubBackward1>),
 tensor(3.9432, grad_fn=<MeanBackward0>))

In [47]:
loss_one, loss_two, loss

(tensor([[2.1195, 1.8392],
         [1.9148, 1.8425]], grad_fn=<RsubBackward1>),
 tensor([[1.9155, 2.1013],
         [1.7246, 2.0088]], grad_fn=<RsubBackward1>),
 tensor(1.9716, grad_fn=<MeanBackward0>))

In [38]:
loss

tensor(3.9432, grad_fn=<MeanBackward0>)

In [27]:
loss_one

tensor([[2.1195, 1.8392],
        [1.9148, 1.8425]], grad_fn=<RsubBackward1>)

In [13]:
online_pred_one.shape, online_pred_two.shape

(torch.Size([2, 256]), torch.Size([2, 256]))

In [16]:
loss_one

tensor([2.1195, 1.8425], grad_fn=<RsubBackward1>)

In [17]:
x = online_pred_one
y = target_proj_two.detach()

In [24]:
(x * y).sum(dim=-1)

tensor([-2.0943,  3.2139], grad_fn=<SumBackward1>)

In [23]:
torch.matmul(x, y.T)

tensor([[-2.0943,  2.6701],
        [ 1.8346,  3.2139]], grad_fn=<MmBackward0>)